# Lab 3: LangGraph Workflows

**Objective:** Build stateful agent workflows with conditional routing and human-in-the-loop patterns.

**Duration:** ~45 minutes

**What You'll Learn:**
- How to create state graphs with LangGraph
- How to implement conditional routing
- How to add human-in-the-loop approval
- How to use checkpoints for persistence

## Part 1: Setup and Introduction

LangGraph allows you to build agent workflows as directed graphs, giving you fine-grained control over execution flow.

In [ ]:
# Environment setup
import os
from dotenv import load_dotenv

load_dotenv()

assert os.getenv("OPENAI_API_KEY"), "OPENAI_API_KEY not found"
print("Environment configured!")

In [ ]:
# Core imports
from typing import TypedDict, Annotated, Literal, cast, Any
from langgraph.graph import StateGraph, END, START
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
import operator

print("Imports successful!")

## Part 2: Building a Simple Graph

Let's start with a simple linear workflow to understand the basics.

In [ ]:
# Define the state schema
class SimpleState(TypedDict):
    """State that flows through our graph."""
    input: str
    processed: str
    output: str

print("State schema defined!")

In [ ]:
# Define node functions
def process_input(state: SimpleState) -> dict:
    """First node: Process the input."""
    input_text = state["input"]
    processed = input_text.upper()  # Simple transformation
    print(f"[process_input] Processing: {input_text} -> {processed}")
    return {"processed": processed}

def generate_output(state: SimpleState) -> dict:
    """Second node: Generate final output."""
    processed = state["processed"]
    output = f"Result: {processed} (length: {len(processed)})"
    print(f"[generate_output] Creating output: {output}")
    return {"output": output}

print("Node functions defined!")

In [ ]:
# Build the graph
simple_graph = StateGraph(SimpleState)

# Add nodes
simple_graph.add_node("process", process_input)
simple_graph.add_node("generate", generate_output)

# Add edges
simple_graph.add_edge(START, "process")
simple_graph.add_edge("process", "generate")
simple_graph.add_edge("generate", END)

# Compile the graph
simple_app = simple_graph.compile()

print("Graph compiled!")

In [ ]:
# Run the graph
# Construct a correctly-typed SimpleState before invoking to satisfy type checkers
input_state = cast(SimpleState, {"input": "hello world", "processed": "", "output": ""})
result = simple_app.invoke(input_state)

print("\nFinal State:")
for key, value in result.items():
    print(f"  {key}: {value}")

## Part 3: Conditional Routing

Now let's add conditional routing based on the content.

In [ ]:
# Enhanced state with routing information
class RoutingState(TypedDict):
    """State with routing capability."""
    input: str
    category: str
    result: str

print("Routing state defined!")

In [ ]:
# Node functions for routing
def classify_input(state: RoutingState) -> dict:
    """Classify the input to determine routing."""
    input_text = state["input"].lower()
    
    if any(word in input_text for word in ["calculate", "math", "sum", "multiply"]):
        category = "math"
    elif any(word in input_text for word in ["search", "find", "look up"]):
        category = "search"
    else:
        category = "general"
    
    print(f"[classify] Input classified as: {category}")
    return {"category": category}

def handle_math(state: RoutingState) -> dict:
    """Handle math-related queries."""
    print("[handle_math] Processing math query")
    return {"result": "Math handler: I would calculate the result here."}

def handle_search(state: RoutingState) -> dict:
    """Handle search-related queries."""
    print("[handle_search] Processing search query")
    return {"result": "Search handler: I would search for information here."}

def handle_general(state: RoutingState) -> dict:
    """Handle general queries."""
    print("[handle_general] Processing general query")
    return {"result": "General handler: I would provide a general response here."}

print("Handler functions defined!")

In [ ]:
# Routing function
def route_query(state: RoutingState) -> Literal["math", "search", "general"]:
    """Determine which handler to use based on category."""
    category = state["category"]
    print(f"[router] Routing to: {category}")
    # Cast the runtime string to the Literal type for the type checker
    return cast(Literal["math", "search", "general"], category)

print("Router function defined!")

In [ ]:
# Build the routing graph
routing_graph = StateGraph(RoutingState)

# Add nodes
routing_graph.add_node("classify", classify_input)
routing_graph.add_node("math", handle_math)
routing_graph.add_node("search", handle_search)
routing_graph.add_node("general", handle_general)

# Add edges
routing_graph.add_edge(START, "classify")

# Add conditional routing
routing_graph.add_conditional_edges(
    "classify",
    route_query,
    {
        "math": "math",
        "search": "search",
        "general": "general"
    }
)

# All handlers go to END
routing_graph.add_edge("math", END)
routing_graph.add_edge("search", END)
routing_graph.add_edge("general", END)

# Compile
routing_app = routing_graph.compile()

print("Routing graph compiled!")

In [ ]:
# Test different inputs
test_inputs = [
    "Calculate 5 + 3",
    "Search for Python tutorials",
    "Tell me a joke"
]

for test_input in test_inputs:
    print(f"\n{'='*50}")
    print(f"Input: {test_input}")
    # Construct a properly-typed RoutingState before invoking
    state = cast(RoutingState, {"input": test_input, "category": "", "result": ""})
    result = routing_app.invoke(state)
    print(f"Result: {result['result']}")

## Part 4: Building an LLM-Powered Agent

Now let's build a real agent with LLM integration.

In [ ]:
# Initialize LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

print("LLM initialized!")

In [ ]:
# Agent state with messages
class AgentState(TypedDict):
    """State for our LLM agent."""
    messages: Annotated[list, operator.add]  # Messages accumulate
    task: str
    plan: str
    result: str
    status: str

print("Agent state defined!")

In [ ]:
# Agent nodes
def planner(state: AgentState) -> dict:
    """Create a plan for the task."""
    task = state["task"]
    
    messages = [
        SystemMessage(content="You are a planning assistant. Create a brief 3-step plan."),
        HumanMessage(content=f"Create a plan for: {task}")
    ]
    
    response = llm.invoke(messages)
    plan = response.content
    
    print(f"[planner] Created plan")
    return {
        "plan": plan,
        "messages": [AIMessage(content=f"Plan: {plan}")],
        "status": "planned"
    }

def executor(state: AgentState) -> dict:
    """Execute the plan."""
    plan = state["plan"]
    task = state["task"]
    
    messages = [
        SystemMessage(content="You are an execution assistant. Execute the plan and provide results."),
        HumanMessage(content=f"Task: {task}\nPlan: {plan}\n\nExecute this plan and provide the result.")
    ]
    
    response = llm.invoke(messages)
    result = response.content
    
    print(f"[executor] Executed plan")
    return {
        "result": result,
        "messages": [AIMessage(content=f"Execution result: {result}")],
        "status": "executed"
    }

def reviewer(state: AgentState) -> dict:
    """Review and finalize the result."""
    result = state["result"]
    task = state["task"]
    
    messages = [
        SystemMessage(content="You are a quality reviewer. Review the result and provide final assessment."),
        HumanMessage(content=f"Original task: {task}\nResult: {result}\n\nProvide a brief quality assessment.")
    ]
    
    response = llm.invoke(messages)
    
    print(f"[reviewer] Reviewed result")
    return {
        "messages": [AIMessage(content=f"Review: {response.content}")],
        "status": "reviewed"
    }

print("Agent nodes defined!")

In [ ]:
# Build the agent graph
agent_graph = StateGraph(AgentState)

# Add nodes
agent_graph.add_node("planner", planner)
agent_graph.add_node("executor", executor)
agent_graph.add_node("reviewer", reviewer)

# Add edges (linear flow)
agent_graph.add_edge(START, "planner")
agent_graph.add_edge("planner", "executor")
agent_graph.add_edge("executor", "reviewer")
agent_graph.add_edge("reviewer", END)

# Compile with memory
memory = MemorySaver()
agent_app = agent_graph.compile(checkpointer=memory)

print("Agent graph compiled!")

In [ ]:
# Run the agent
config = cast(Any, {"configurable": {"thread_id": "demo-1"}})

# Create a properly-typed AgentState for invocation
input_state = cast(AgentState, {
    "task": "Write a haiku about programming",
    "messages": [HumanMessage(content="Starting task...")],
    "plan": "",
    "result": "",
    "status": "",
})

result = agent_app.invoke(input_state, config)

print("\n" + "="*50)
print("FINAL RESULT:")
print("="*50)
print(f"\nPlan:\n{result['plan'][:200]}...")
print(f"\nResult:\n{result['result'][:300]}...")
print(f"\nStatus: {result['status']}")

## Part 5: Adding Human-in-the-Loop

Let's add an approval step where a human can review before execution.

In [ ]:
# State with approval
class ApprovalState(TypedDict):
    """State with human approval."""
    task: str
    plan: str
    approved: bool
    result: str
    feedback: str

print("Approval state defined!")

In [ ]:
def create_plan(state: ApprovalState) -> dict:
    """Create a plan for approval."""
    task = state["task"]
    
    messages = [
        SystemMessage(content="Create a detailed plan for the task."),
        HumanMessage(content=f"Task: {task}")
    ]
    
    response = llm.invoke(messages)
    print(f"[create_plan] Plan created, awaiting approval")
    
    return {"plan": response.content}

def execute_plan(state: ApprovalState) -> dict:
    """Execute the approved plan."""
    plan = state["plan"]
    
    messages = [
        SystemMessage(content="Execute the plan and provide results."),
        HumanMessage(content=f"Execute this plan: {plan}")
    ]
    
    response = llm.invoke(messages)
    print(f"[execute_plan] Plan executed")
    
    return {"result": response.content}

def revise_plan(state: ApprovalState) -> dict:
    """Revise the plan based on feedback."""
    plan = state["plan"]
    feedback = state["feedback"]
    
    messages = [
        SystemMessage(content="Revise the plan based on the feedback."),
        HumanMessage(content=f"Original plan: {plan}\nFeedback: {feedback}\nCreate revised plan.")
    ]
    
    response = llm.invoke(messages)
    print(f"[revise_plan] Plan revised based on feedback")
    
    return {"plan": response.content, "approved": False}

print("Approval functions defined!")

In [ ]:
# Routing function for approval
def check_approval(state: ApprovalState) -> Literal["execute", "revise"]:
    """Route based on approval status."""
    if state.get("approved", False):
        print("[router] Plan approved -> executing")
        return "execute"
    else:
        print("[router] Plan not approved -> revising")
        return "revise"

print("Approval router defined!")

In [ ]:
# Build approval graph
approval_graph = StateGraph(ApprovalState)

# Add nodes
approval_graph.add_node("plan", create_plan)
approval_graph.add_node("execute", execute_plan)
approval_graph.add_node("revise", revise_plan)

# Add edges
approval_graph.add_edge(START, "plan")

# Conditional edge after planning - this is where human approval happens
approval_graph.add_conditional_edges(
    "plan",
    check_approval,
    {
        "execute": "execute",
        "revise": "revise"
    }
)

# After revision, go back to check approval
approval_graph.add_conditional_edges(
    "revise",
    check_approval,
    {
        "execute": "execute",
        "revise": "revise"
    }
)

approval_graph.add_edge("execute", END)

# Compile with interrupt for human approval
approval_memory = MemorySaver()
approval_app = approval_graph.compile(
    checkpointer=approval_memory,
    interrupt_before=["execute", "revise"]  # Pause before these nodes
)

print("Approval graph compiled with interrupts!")

In [ ]:
# Start the workflow
config = cast(Any, {"configurable": {"thread_id": "approval-demo"}})

# First run - creates the plan and pauses
initial_state = cast(ApprovalState, {"task": "Create a marketing email for a new product launch", "plan": "", "approved": False, "result": "", "feedback": ""})
result = approval_app.invoke(initial_state, config)

print("\n" + "="*50)
print("PLAN CREATED - AWAITING APPROVAL")
print("="*50)
print(f"\nPlan:\n{result['plan']}")

In [ ]:
# Simulate human approval
print("Simulating human approval...")

# Update state with approval (cast config to Any for the type checker)
approval_app.update_state(cast(Any, config), {"approved": True})  # Human approves the plan

# Continue execution
result = approval_app.invoke(None, cast(Any, config))

print("\n" + "="*50)
print("EXECUTION COMPLETE")
print("="*50)
print(f"\nResult:\n{result.get('result', 'No result')}")

## Part 6: Adding Tools to LangGraph

Let's build an agent with tool-calling capabilities.

In [ ]:
from langchain.tools import tool
from langgraph.prebuilt import ToolNode, tools_condition

# Define tools
@tool
def search(query: str) -> str:
    """Search for information on a topic."""
    return f"Search results for '{query}': Found 3 relevant articles about {query}."

@tool  
def calculator(expression: str) -> str:
    """Calculate a mathematical expression."""
    try:
        result = eval(expression)
        return f"Result: {result}"
    except:
        return "Error: Could not calculate"

@tool
def get_weather(city: str) -> str:
    """Get weather for a city."""
    import random
    temp = random.randint(60, 85)
    return f"Weather in {city}: {temp}°F, sunny"

tools = [search, calculator, get_weather]
print(f"Defined {len(tools)} tools!")

In [ ]:
# State for tool-using agent
class ToolAgentState(TypedDict):
    messages: Annotated[list, operator.add]

# Bind tools to LLM
llm_with_tools = llm.bind_tools(tools)

# Agent node
def agent_node(state: ToolAgentState) -> dict:
    """Agent decides what to do."""
    messages = state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

print("Tool agent configured!")

In [ ]:
# Build tool agent graph
tool_graph = StateGraph(ToolAgentState)

# Add nodes
tool_graph.add_node("agent", agent_node)
tool_graph.add_node("tools", ToolNode(tools))

# Add edges
tool_graph.add_edge(START, "agent")

# Conditional edge: if agent calls tools, go to tools node; else end
tool_graph.add_conditional_edges(
    "agent",
    tools_condition  # Built-in condition for tool calling
)

# Tools always go back to agent
tool_graph.add_edge("tools", "agent")

# Compile
tool_agent = tool_graph.compile()

print("Tool agent compiled!")

In [ ]:
# Test the tool agent
result = tool_agent.invoke({
    "messages": [HumanMessage(content="What's the weather in Seattle and calculate 25 * 4?")]
})

print("\n" + "="*50)
print("AGENT RESPONSE:")
print("="*50)
for msg in result["messages"]:
    print(f"\n{type(msg).__name__}:")
    if hasattr(msg, 'content') and msg.content:
        print(msg.content[:500])

## Challenge: Build a Document Processing Workflow

Create a LangGraph workflow that:

1. **Classifies** a document type (email, report, memo)
2. **Extracts** key information based on type
3. **Summarizes** the content
4. **Routes** to different handlers based on urgency

In [ ]:
# TODO: Define the state
class DocumentState(TypedDict):
    document: str
    doc_type: str  # email, report, memo
    extracted_info: dict
    summary: str
    urgency: str  # high, medium, low
    response: str

# TODO: Define node functions
# def classify_document(state) -> dict:
#     pass

# def extract_info(state) -> dict:
#     pass

# def summarize(state) -> dict:
#     pass

# def handle_urgent(state) -> dict:
#     pass

# def handle_normal(state) -> dict:
#     pass

# TODO: Build the graph with conditional routing

In [ ]:
# TODO: Test your document processing workflow
test_documents = [
    "URGENT: Server down! Need immediate action. Production database unreachable.",
    "Weekly Report: Sales up 15% this quarter. Team performing well.",
    "Meeting request: Let's discuss the Q2 roadmap next Tuesday."
]

## Lab Summary

In this lab, you learned:

1. **State Management**: Using TypedDict to define state that flows through graphs
2. **Graph Building**: Creating nodes, edges, and conditional routing
3. **Conditional Routing**: Directing flow based on state
4. **Human-in-the-Loop**: Using interrupts for approval workflows
5. **Tool Integration**: Adding tools to LangGraph agents

### Key Takeaways

- LangGraph gives you **fine-grained control** over agent workflows
- **State** is explicitly defined and flows through the graph
- **Conditional edges** enable dynamic routing
- **Checkpointing** enables persistence and human-in-the-loop
- **Tool integration** works seamlessly with the prebuilt `ToolNode`

### When to Use LangGraph

| Scenario | Use LangGraph? |
|----------|---------------|
| Simple Q&A | No, use basic chains |
| Complex workflow | Yes |
| Need approval steps | Yes |
| Cycles/loops needed | Yes |
| State persistence | Yes |
| Simple tool calling | Maybe, basic agents work too |

### Next Steps

In Lab 4, you'll build your own capstone agent using any framework!

In [ ]:
print("Lab 3 complete!")
print("\nYou now know how to build stateful agent workflows with LangGraph.")